<a href="https://colab.research.google.com/github/yapyang040527/PLanguage/blob/main/HW5_%E5%8D%88%E9%A4%90_%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
!pip install folium

In [201]:
import folium
lat = 25.02580828805888
ing = 121.52854685273857
m = folium.Map(location=(lat, ing), zoom_start=20)

In [202]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [203]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [204]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "餐廳列表"
TIMEZONE = "Asia/Taipei"

In [205]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,店家名稱,類別/招牌品項,參考地點 (捷運站/商圈),備註/特色,lat,lng
0,溫州街蘿蔔絲餅達人,蘿蔔絲餅、豆沙餅,捷運台電大樓站/溫州街,經典排隊銅板點心，蘿蔔絲餅外酥內餡飽滿。,25.02625,121.5303
1,瑞安豆漿大王,傳統中式早餐/宵夜,捷運大安站/瑞安街,24小時營業，燒餅油條、鹹豆漿、韭菜盒子等。,25.02853,121.5414
2,樂業麵線,大腸麵線,捷運六張犁站/樂業街,大碗綜合麵線受歡迎，湯頭鮮美。,25.02039,121.5525
3,麗如小吃店,乾意麵、黑白切,捷運大安站/瑞安街,乾意麵、貢丸湯等，價格平實的傳統小吃店。,25.02453,121.5287
4,古早人水煎包,高麗菜水煎包、韭菜水煎包,大安森林公園周邊,麵皮脆酥，內餡飽滿，銅板價早餐/點心。,25.03292,121.5382


In [206]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "模擬資料"
TIMEZONE = "Asia/Taipei"

In [207]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,餐廳名稱,價位,距離,是否營業,特色描述
0,雙月食品社,$,短,是,米其林雞湯/中式
1,溫州街蘿蔔絲餅,$,短,是,經典點心/便宜
2,杭州小籠湯包,$$,中,是,平價小籠包
3,瑞安豆漿大王,$,短,否,24H中式早餐
4,不二煮藝,$$,中,是,平價義式/健康


In [208]:
# 假設您的 DataFrame 實際欄位名稱是 'latitude' 和 'longitude'
# 請根據您實際執行的 print(df_restaurants.columns) 結果來調整以下欄位名稱

# ----------------- 請替換以下變數為實際欄位名稱 -----------------
LATITUDE_COL = 'lat'          # 請替換為您 DataFrame 中緯度欄位的正確名稱 (例如: 'Latitude')
LONGITUDE_COL = 'lng'         # 請替換為您 DataFrame 中經度欄位的正確名稱 (例如: 'Longitude')
TOOLTIP_COL = '店家名稱'      # 請替換為您 DataFrame 中店家名稱的正確名稱
POPUP_COL = '類別/招牌品項'   # 請替換為您 DataFrame 中類別的正確名稱
# --------------------------------------------------------------

# 檢查欄位是否存在，如果不存在則跳出迴圈
required_cols = [LATITUDE_COL, LONGITUDE_COL, TOOLTIP_COL, POPUP_COL]
missing_cols = [col for col in required_cols if col not in df_restaurants.columns]

if missing_cols:
    print(f"🚨 錯誤：DataFrame 中缺少以下欄位：{missing_cols}")
else:
    print("✅ 欄位檢查通過，開始繪製地圖標記...")

    # 使用 iterrows() 代替 range(len(df))，這是更推薦的 Pandas 遍歷方式
    for index, row in df_restaurants.iterrows():
        try:
            # 確保數據類型轉換成功
            lat = float(row[LATITUDE_COL])
            lng = float(row[LONGITUDE_COL])

            tooltip_text = row[TOOLTIP_COL]
            popup_text = row[POPUP_COL]

            folium.Marker(
                location = [lat, lng],
                tooltip = tooltip_text,
                popup = popup_text,
                # 建議使用更符合餐廳的圖示
                icon=folium.Icon(icon="cutlery", prefix='fa', color="darkgreen")
            ).add_to(m)

        except ValueError:
            # 處理欄位中包含非數字字串導致 float() 轉換失敗的情況
            print(f"⚠️ 警告：索引 {index} 處的經緯度值無法轉換為數字，跳過該項。")
        except Exception as e:
             print(f"❌ 繪製標記時發生其他錯誤於索引 {index}：{e}")

# --------------------------------------------------------------

🚨 錯誤：DataFrame 中缺少以下欄位：['lat', 'lng', '店家名稱', '類別/招牌品項']


In [209]:
m

In [218]:
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import folium
from io import BytesIO
from datetime import datetime
import sys

# --- Google Colab/Google Sheets 認證與數據載入 ---
try:
    from google.colab import auth
    import gspread
    from google.auth import default
    from gspread_dataframe import get_as_dataframe
    SHEETS_LOADED = False
    SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
    WORKSHEET_NAME = "餐廳列表"
except ImportError:
    print("⚠️ 警告：Google Colab/Gspread 相關套件未安裝，將使用靜態備份數據。")

# ------------------------------------------------------------------
# 1. 數據初始化：全域變數聲明與數據載入
# ------------------------------------------------------------------
global df_places

# 靜態備份數據 (使用您 Sheets 中已有的欄位，並添加 User_Score)
STATIC_BACKUP_DATA = {
    '地點名稱': [
        '真劍拉麵 (師大)', '糊塗麵 (溫州街)', '鐵支涮火鍋 (和平東)', '薩莉亞義式餐廳 (師大)', '永豐盛饅頭 (師大路)', '燈籠滷味', '金雞園 (港式)', 'Subway (師大店)', '許記生煎包', '溫州大餛飩', '滇味廚房 (雲南菜)', '韓江烤肉 (吃到飽)'
    ],
    '特色描述': [
        '拉麵', '麵食', '火鍋', '義式', '點心', '滷味', '港式', '輕食', '小吃', '麵食', '雲南菜', '韓式烤肉'
    ],
    'lat': [25.0255, 25.0270, 25.0275, 25.0260, 25.0245, 25.0270, 25.0285, 25.0265, 25.0248, 25.0272, 25.0268, 25.0295],
    'lng': [121.5295, 121.5300, 121.5290, 121.5280, 121.5275, 121.5305, 121.5285, 121.5285, 121.5302, 121.5292, 121.5308, 121.5288]
}

# 輔助數據：用於在 Sheets 中缺少時，程式內部仍需要這些欄位來顯示表格
FALLBACK_COLUMNS = {
    '價位': '$',
    '距離': '中',
    '是否營業': '是',
    'Wait_Time_min': 15
}

# ------------------------------------------------------------------
# 核心載入邏輯 - 修正版本 (只依賴已確認存在的欄位)
# ------------------------------------------------------------------

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    gsheets = gc.open_by_url(SHEET_URL)
    sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
    df_places = pd.DataFrame(sheets[1:], columns=sheets[0])

    # ----------------- 欄位名稱映射與清理 -----------------
    MAPPING = {
        '店家名稱': '地點名稱',
        '類別/招牌品項': '特色描述',
        # !!! ⚠️ 刪除所有關於 '價位', '距離', '是否營業' 的映射，因為它們不存在 !!!
    }

    # 執行映射
    df_places = df_places.rename(columns=MAPPING)

    # 增加 Fallback 欄位 (讓表格可以顯示)
    for col, default_val in FALLBACK_COLUMNS.items():
        if col not in df_places.columns:
            df_places[col] = default_val if col != 'Wait_Time_min' else np.random.randint(5, 30, size=len(df_places))

    # 確保經緯度是 float 類型
    df_places['lat'] = pd.to_numeric(df_places['lat'], errors='coerce')
    df_places['lng'] = pd.to_numeric(df_places['lng'], errors='coerce')

    # 初始化 Gradio 邏輯所需的 User_Score 欄位
    if 'User_Score' not in df_places.columns:
        df_places['User_Score'] = 0
    df_places['User_Score'] = pd.to_numeric(df_places['User_Score'], errors='coerce').fillna(0).astype(int)

    # 處理空值：移除經緯度缺失的行
    df_places.dropna(subset=['lat', 'lng'], inplace=True)
    df_places_count = len(df_places)
    SHEETS_LOADED = True

    if df_places_count == 0:
        raise Exception("載入地點數為零，數據可能不完整。")

    print(f"✅ 成功從 Google Sheets 載入數據。共 {df_places_count} 筆地點。")

except Exception as e:
    print(f"❌ Google Sheets 載入失敗，將使用靜態備份數據。錯誤: {e}")
    df_places = pd.DataFrame(STATIC_BACKUP_DATA)
    df_places['User_Score'] = 0
    df_places['價位'] = '$' * np.random.randint(1, 4, size=len(df_places))
    df_places['距離'] = np.random.choice(['短', '中', '長'], size=len(df_places))
    df_places['是否營業'] = np.random.choice(['是', '否'], size=len(df_places))
    df_places['Wait_Time_min'] = np.random.randint(5, 30, size=len(df_places))
    df_places_count = len(df_places)
    print("已啟用靜態備份數據。")

# ------------------------------------------------------------------
# 2. 核心功能函式 (僅保留特色描述篩選)
# ------------------------------------------------------------------

def calculate_wait_time(df):
    """簡化：使用隨機數填充等待時間，避免依賴不存在的 '距離' 欄位"""
    if 'Wait_Time_min' not in df.columns:
         df['Wait_Time_min'] = np.random.randint(5, 30, size=len(df))
    return df

def rate_place(place_name, action):
    """根據使用者動作更新地點的偏好分數 (保持不變)"""
    global df_places
    match = df_places['地點名稱'] == place_name
    if not match.any():
        return f"錯誤：找不到地點 '{place_name}'。", None

    if action == "Like":
        df_places.loc[match, 'User_Score'] += 1
    elif action == "Dislike":
        current_score = df_places.loc[match, 'User_Score'].iloc[0]
        df_places.loc[match, 'User_Score'] = max(0, current_score - 1)

    message = f"✅ 已更新 '{place_name}' 的偏好。當前分數：{df_places.loc[match, 'User_Score'].iloc[0]}"
    updated_score_df = df_places[['地點名稱', 'User_Score']].sort_values(by='User_Score', ascending=False)
    return message, updated_score_df

# *** 篩選功能：僅能使用特色描述 ***
def filter_and_recommend(df, category_filter):
    """根據特色描述和偏好分數，加權隨機推薦 3 個地點"""

    category_list = category_filter if category_filter else df['特色描述'].unique().tolist()

    filtered_df = df[
        df['特色描述'].isin(category_list)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "", ""

    n_recommend = min(3, len(filtered_df))
    weights = filtered_df['User_Score'] + 1

    recommended_df = filtered_df.sample(n=n_recommend, replace=False, weights=weights)

    recommend_list = "\n".join([
        f"{i+1}. {row['地點名稱']} ({row['價位']} / {row['距離']}) - 預估等待 {row['Wait_Time_min']} 分鐘 - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['地點名稱']}，特色：{row['特色描述']}，預估等待{row['Wait_Time_min']}分鐘"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

async def generate_summary(ai_input):
    if not ai_input: return "請先點擊「開始推薦」以產生地點清單。"
    await asyncio.sleep(1)

    mock_summary = f"""
今天的選擇是：{'、'.join([r.split('，')[0] for r in ai_input.split('、')])}。
您的決策考量應為：
1. **效率考量：** 留意「預估等待時間」，選擇等待時間最短的地點。
2. **偏好考量：** 推薦清單中已經優先選擇了您過去喜歡的選項。
3. **綜合建議：** 請根據您對 {', '.join(df_places['特色描述'].unique())} 的偏好進行最終決定。
"""
    return mock_summary.strip()

def create_folium_map(recommended_df):
    if recommended_df.empty:
        return "<p style='color:#90caf9; text-align:center;'>請先點擊「開始推薦」以顯示地圖。</p>"

    center_lat = recommended_df['lat'].mean()
    center_lng = recommended_df['lng'].mean()
    map_zoom = 13 if len(recommended_df) > 1 else 15
    m = folium.Map(location=[center_lat, center_lng], zoom_start=map_zoom, tiles="cartodbdarkmatter")

    for _, row in recommended_df.iterrows():
        # 使用 Fallback 欄位進行顯示
        popup_html = f"<b>{row['地點名稱']}</b><br>類別: {row['特色描述']}<br>價位: {row.get('價位', 'N/A')}<br>距離: {row.get('距離', 'N/A')}<br>預估等待: {row.get('Wait_Time_min', 'N/A')} 分鐘"
        folium.Marker(
            location=[row['lat'], row['lng']],
            tooltip=row['地點名稱'],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(icon="cutlery", prefix='fa', color="darkgreen")
        ).add_to(m)

    data = BytesIO()
    m.save(data, close_file=False)
    return data.getvalue().decode('utf8')

async def wrapper_recommend(category_filter):
    global df_places
    df_temp = calculate_wait_time(df_places.copy())
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_temp, category_filter
    )
    ai_summary = await generate_summary(ai_input)
    map_html = create_folium_map(recommended_df)

    # 確保輸出 dataframe 包含所有需要的欄位
    display_df = recommended_df[['地點名稱','價位','距離', 'Wait_Time_min', 'User_Score']].reset_index(drop=True)

    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的地點。", "（無建議）", map_html

    return display_df, recommend_list, ai_summary, map_html

# -----------------------
# 3. CSS 美化 (保持不變)
# -----------------------
custom_css = """
/* ... (CSS 代碼保持不變) ... */
.auto-height-textbox textarea {
    min-height: 150px !important;
    max-height: 500px !important;
    height: auto !important;
    overflow-y: auto !important;
    border-radius: 12px !important;
    padding: 12px !important;
    border: 1px solid #444 !important;
    background-color: #2c2c2c !important;
    color: #f5f5f5 !important;
    font-size: 15px !important;
    box-shadow: 0 2px 6px rgba(0,0,0,0.3) !important;
    transition: all 0.3s ease !important;
}
"""

# -----------------------
# 4. Gradio 介面配置 (大改動：只顯示特色描述篩選)
# -----------------------
with gr.Blocks(title="師大地點決策器", css=custom_css, theme=gr.themes.Base()) as demo:
    source_text = "Google Sheets" if 'SHEETS_LOADED' in locals() and SHEETS_LOADED else "靜態備份"

    gr.Markdown(
        f"""
        # 🍽️ 師大周邊地點決策器 (基於現有數據)
        ### 數據來源：{source_text}。共 **{df_places_count}** 筆地點。
        ---
        """
    )
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選 (僅可按『特色描述』篩選)")

            # 使用 '特色描述' 進行篩選
            category_select = gr.CheckboxGroup(
                df_places['特色描述'].unique().tolist(),
                label="類別/特色過濾 (可複選)",
                value=df_places['特色描述'].unique().tolist()
            )

            # 隱藏價位/距離/營業狀態篩選器
            # price_select = gr.HTML(label="價位", value="<p>您的數據中缺少『價位』欄位。</p>")
            # distance_select = gr.HTML(label="距離", value="<p>您的數據中缺少『距離』欄位。</p>")
            # available_toggle = gr.HTML(label="是否營業", value="<p>您的數據中缺少『是否營業』欄位。</p>")

            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (加權隨機 3 個地點)")
            output_dataframe = gr.DataFrame(
                # 欄位名稱保留，但值為隨機/預設值
                headers=["地點名稱","價位","距離", "預估等待 (分)", "偏好分數"],
                datatype=["str","str","str", "number", "number"]
            )
            output_list = gr.Textbox(label="推薦地點細節 (包含等待時間)", interactive=False, elem_classes="auto-height-textbox")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## ⭐ 偏好回饋 (影響下次推薦權重)")
            rating_name_input = gr.Textbox(label="輸入地點名稱以回饋", placeholder="例如：燈籠滷味")
            with gr.Row():
                like_btn = gr.Button("👍 增加偏好 (+1)")
                dislike_btn = gr.Button("👎 降低偏好 (-1)")
            feedback_message = gr.Textbox(label="操作狀態", interactive=False)

        with gr.Column(scale=1):
             gr.Markdown("## 💾 當前地點偏好分數一覽")
             score_display_df = gr.Dataframe(
                 headers=["地點名稱", "偏好分數"],
                 datatype=["str", "number"],
                 value=df_places[['地點名稱', 'User_Score']].sort_values(by='User_Score', ascending=False),
                 interactive=False
             )

    gr.Markdown("## 📍 推薦地點地圖定位")
    output_map = gr.HTML(label="互動式地圖", value="<p style='color:#90caf9; text-align:center;'>地圖將於點擊推薦後顯示。</p>", elem_classes="auto-height-textbox")

    gr.Markdown("## 🤖 AI 決策建議")
    output_summary = gr.Textbox(label="AI 決策顧問報告", interactive=False, elem_classes="auto-height-textbox")

    # 綁定事件
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[category_select], # 僅傳入類別篩選
        outputs=[output_dataframe, output_list, output_summary, output_map]
    )
    like_btn.click(
        fn=lambda name: rate_place(name, "Like"), inputs=[rating_name_input], outputs=[feedback_message, score_display_df]
    )
    dislike_btn.click(
        fn=lambda name: rate_place(name, "Dislike"), inputs=[rating_name_input], outputs=[feedback_message, score_display_df]
    )

# -----------------------
# 5. 運行 Gradio
# -----------------------
if __name__ == "__main__":
    demo.launch(inbrowser=True)

✅ 成功從 Google Sheets 載入數據。共 12 筆地點。
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5edf2396f35f16980.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
